In [4]:
import yfinance as yf
import os 
import pandas as pd 
import numpy as np
import csv



In [ ]:
"""url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
sp500_tickers = sp500_table['Symbol'].tolist()"""
sp500_tickers = ["TSLA"]


In [6]:
total_data = []
folder = "data"
data_type = "YF"
for ticker in sp500_tickers:
    file = f"{ticker} {data_type} Data.csv"
    file_path = os.path.join(folder, file)
    if os.path.exists(file_path):
        print(f"file for {ticker} already exists. Skipping...")
        continue
    try:
        df = yf.download(ticker, period="max", progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        if df.empty:
            continue
        df.reset_index(inplace=True)
        
        file = f"{ticker} {data_type} Data.csv"
        df.to_csv(file, index=False)
        print(df.head())
        print(df.tail())
    except Exception as e:
        print(f"error, {e}")


Price       Date  Close   High    Low   Open    Volume
0     2019-09-26  25.76  27.98  24.75  27.17  47067700
1     2019-09-27  25.24  26.17  23.75  26.17  15976000
2     2019-09-30  25.10  25.85  25.00  25.26   4482900
3     2019-10-01  22.51  25.17  22.51  25.00   8816100
4     2019-10-02  22.60  23.18  21.76  22.25   9454700
Price       Date  Close   High    Low   Open    Volume
1459  2025-07-18  6.290  6.410  6.232  6.390   7579200
1460  2025-07-21  6.190  6.375  6.160  6.305   6420700
1461  2025-07-22  6.490  6.670  6.165  6.200  11253300
1462  2025-07-23  6.770  7.160  6.515  6.515  13512700
1463  2025-07-24  6.465  6.710  6.460  6.650   4419073
